In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import cv2
import numpy as np
import tensorflow as tf

# Set the path to the directory and image name
image_path = "/kaggle/input/ripeness-detection-of-mango/Stage_0 (Unripe)/Test/IMG20200713145725.jpg"

# Preprocess the image
def preprocess_image(image_path, target_size=(224, 224)):
    # Load the image
    image = cv2.imread(image_path)
    # Resize to MobileNetV2 input size
    image_resized = cv2.resize(image, target_size)
    # Normalize to [0, 1]
    image_normalized = image_resized / 255.0
    # Add batch dimension
    image_expanded = np.expand_dims(image_normalized, axis=0)
    return image_expanded

# Preprocess the input image
input_image = preprocess_image(image_path)


In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
import os
import numpy as np
import matplotlib.pyplot as plt


In [8]:
base_dir = '/kaggle/input/ripeness-detection-of-mango'

# Define image dimensions and batch size
IMG_SIZE = (224, 224)  # Required input size for MobileNetV2
BATCH_SIZE = 32


In [9]:
# Training data generator
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80-20 split for train-validation
)

# Training data loader
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# Validation data loader
val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 763 images belonging to 4 classes.
Found 188 images belonging to 4 classes.


In [10]:
# Load MobileNetV2 without the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
base_model.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output_layer = Dense(train_generator.num_classes, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display model summary
model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,468 (9.24 MB)

 Trainable params: 164,484 (642.52 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [11]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,  # Start with fewer epochs to test
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=val_generator.samples // BATCH_SIZE
)


Epoch 1/25


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1734529775.952624     117 service.cc:145] XLA service 0x7fb174002740 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734529775.952687     117 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1734529775.952691     117 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


 1/23 ━━━━━━━━━━━━━━━━━━━━ 5:09 14s/step - accuracy: 0.2812 - loss: 1.3367

I0000 00:00:1734529782.069412     117 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


23/23 ━━━━━━━━━━━━━━━━━━━━ 30s 738ms/step - accuracy: 0.5176 - loss: 1.0113 - val_accuracy: 0.6562 - val_loss: 0.8306
Epoch 2/25
 1/23 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.7812 - loss: 0.5172

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 192ms/step - accuracy: 0.7812 - loss: 0.5172 - val_accuracy: 0.8214 - val_loss: 0.4702
Epoch 3/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 345ms/step - accuracy: 0.8132 - loss: 0.4897 - val_accuracy: 0.7812 - val_loss: 0.7249
Epoch 4/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7500 - loss: 0.6072 - val_accuracy: 0.7857 - val_loss: 0.5375
Epoch 5/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 346ms/step - accuracy: 0.8331 - loss: 0.3705 - val_accuracy: 0.7750 - val_loss: 0.6383
Epoch 6/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8750 - loss: 0.2629 - val_accuracy: 0.7500 - val_loss: 0.6662
Epoch 7/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 348ms/step - accuracy: 0.8765 - loss: 0.3392 - val_accuracy: 0.8125 - val_loss: 0.5316
Epoch 8/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9375 - loss: 0.2520 - val_accuracy: 0.7143 - val_loss: 1.0918
Epoch 9/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 350ms/step - accuracy: 0.8511 - loss: 0.3391 - val_accuracy: 0.8000 - val

In [12]:
# Evaluate on validation data
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

# Save the model
model.save('mango_ripeness_mobilenetv2.h5')


6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 337ms/step - accuracy: 0.7895 - loss: 0.6751
Validation Accuracy: 80.32%


In [14]:
# Load the saved model
model = tf.keras.models.load_model('mango_ripeness_mobilenetv2.h5')

# Test with a single image
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Path to the test image
test_image_path = '/kaggle/input/ripeness-detection-of-mango/Stage_0 (Unripe)/Test/IMG20200713142258.jpg'

# Preprocess the test image
image = load_img(test_image_path, target_size=IMG_SIZE)
image_array = img_to_array(image) / 255.0
image_array = np.expand_dims(image_array, axis=0)

# Predict ripeness stage
prediction = model.predict(image_array)
predicted_class = np.argmax(prediction)
class_labels = list(train_generator.class_indices.keys())

print(f"Predicted Class: {class_labels[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted Class: Stage_0 (Unripe)


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('mango_ripeness_mobilenetv2.h5')
tflite_model = converter.convert()
with open('mango_ripeness_model.tflite', 'wb') as f:
    f.write(tflite_model)
